In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib
import re
from sklearn import preprocessing

In [2]:
filepath = './tweets.csv'

df = pd.read_csv(filepath, delimiter=',',
                skiprows=1, # skips the first line of the file
                encoding='unicode_escape', # prevents errors on unknown symbols
                usecols=['Vertex 1', 'Vertex 2', 'Tweet'], # only import the columns we use
                )
df.columns = ['source_user', 'mentioned_user', 'tweet_text']

print(df)

         source_user   mentioned_user  \
0           notherle        dude_stay   
1           notherle       facikwacik   
2           notherle     crazycold739   
3           notherle       filippecak   
4           notherle      u_yousolame   
..               ...              ...   
842     bulletking61     bulletking61   
843     mx_hippakrit     mx_hippakrit   
844  shers_wird_stuf  shers_wird_stuf   
845   fastblueapple9  shers_wird_stuf   
846     centurionfox            steam   

                                            tweet_text  
0    RT @dude_stay: Plz help us save the this game ...  
1    RT @FacikWacik: My favorite game is dying ????...  
2    RT @CraZyCold739: I associate with this game n...  
3    RT @FilipPecak: Hi people. Let's save #SafeTF2...  
4    RT @u_yousolame: I been playing tf2 for a few ...  
..                                                 ...  
842              #SafeTF2  PLZZZZZZZZZZZZZZZZZZZZZZZZZ  
843  I know I'm late for this but #SafeTF2 (':\n\nh

In [3]:
# add nodes? first export df to list/dictionary then transform? or easier way?

In [4]:
G = nx.DiGraph()
colors = []
for source_user, mentioned_user, tweet_text in zip(df.source_user, df.mentioned_user, df.tweet_text):

    if source_user != mentioned_user:
        G.add_edge(source_user, mentioned_user)
        if "RT" in tweet_text[:3]:
            colors.append("black")
        else:
            colors.append("red")

node_colors = np.arange(0.0, 1.0, 1/G.number_of_nodes())
print(len(node_colors))

344


In [5]:
G.number_of_nodes()

344

In [6]:
G.number_of_edges()

434

In [7]:
measures = {"centrality": {'in_degree': nx.in_degree_centrality(G), 'out_degree': nx.out_degree_centrality(G),
                           'betweenness': nx.betweenness_centrality(G), 'closeness': nx.closeness_centrality(G)}}
out_degrees = []
for key, value in nx.out_degree_centrality(G).items():
    out_degrees.append(value)

minima = min(out_degrees)
maxima = max(out_degrees)

norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.Greys_r)
node_colors = []
for v in out_degrees:
    node_colors.append(mapper.to_rgba(v)[0])


In [8]:
def print_max(persons):
    max_word = max(persons, key=persons.get)
    print(max_word, persons[max_word])

# Analyze the network
I_words = ["i", "me", "my", "mine"]
you_words = ["you", "yours"]
we_words = ["we", "us", "our", "ours"]

# Calculate per person
persons_i_words = dict()
persons_you_words = dict()
persons_we_words = dict()
tweets_person = dict()
words_person = dict()
for index, row in df.iterrows():
    # count tweets per person
    tweets_person[row['source_user']] = tweets_person.get(row['source_user'], 0) + 1
    
    #split the text on punctuation. 
    split_text = re.split(r'[,;!?:" .\']', row['tweet_text'])
    for word in split_text:
        # count words per person
        words_person[row['source_user']] = words_person.get(row['source_user'], 0) + 1
        
        # count i/you/we words per person
        if word.lower() == 'i':
            persons_i_words[row['source_user']] = persons_i_words.get(row['source_user'], 0) + 1
        elif word.lower() == 'you':
            persons_you_words[row['source_user']] = persons_you_words.get(row['source_user'], 0) + 1
        elif word.lower() == 'we':
            persons_we_words[row['source_user']] = persons_we_words.get(row['source_user'], 0) + 1
print_max(persons_i_words)
print_max(persons_you_words)
print_max(persons_we_words)

# Make a complete dictionary with all scores for analyzing
complete_person = dict()
for person, value in words_person.items():
    complete_person[person] = {}
    complete_person[person]['words'] = value

for person, value in persons_i_words.items():
    complete_person[person]['i_words'] = value
for person, value in persons_you_words.items():
    complete_person[person]['you_words'] = value
for person, value in persons_we_words.items():
    complete_person[person]['we_words'] = value
for person, value in tweets_person.items():
    complete_person[person]['num_tweets'] = value


# Calculate proportions for i/you/we words
for key, value in complete_person.items():   
    if 'i_words' in complete_person[key]:
        complete_person[key]['%_i'] = round(complete_person[key]['i_words']/complete_person[key]['words'], 4)
    else:
        complete_person[key]['%_i'] = 0.0

    if 'you_words' in complete_person[key]:
        complete_person[key]['%_you'] = round(complete_person[key]['you_words']/complete_person[key]['words'], 4)
    else:
        complete_person[key]['%_you'] = 0.0

    if 'we_words' in complete_person[key]:
        complete_person[key]['%_we'] = round(complete_person[key]['we_words']/complete_person[key]['words'], 4)
    else:
        complete_person[key]['%_we'] = 0.0


simon_word 45
simon_word 5
simon_word 9


In [22]:
# Generate linguisticstatus scores
values = []
for key, value in complete_person.items():
    i = complete_person[key]['%_i']
    you = complete_person[key]['%_you']
    we = complete_person[key]['%_we']
    try: 
        lingstat = (you + we) / i
    except ZeroDivisionError: 
        lingstat = you + we
    complete_person[key]['lingstat'] = lingstat
    values.append(lingstat)

# Calculate the normalized lingstat values
mini = min(values)
maxi = max(values)
normalized = []
for value in values:
    x = value/maxi
    if x > 1:
       print(x)
    normalized.append(x)

# Add the normalized lingstat values back to the dictionary, also find user with highest
maxi = 0
max_name = ""
for key in complete_person:
    normal = normalized.pop(0)
    if normal > maxi:
        maxi = normal
        max_name = key
    complete_person[key]['lingstat'] = normal

print(max_name, maxi)

dejdothecringe 1.0


In [20]:
# Other centrality measure
eigenvector = nx.eigenvector_centrality(G)
#print(eigenvector)
# We need to remove the users 'kite__' and 'hotgrossdog', as their scores are way too high in comparison to others
# They both have scores of 0.7, while the next highest score is 8e-05
eigenvector.pop("kite__")
eigenvector.pop("hotgrossdog")
max_eigen = max(eigenvector.values())
for key, value in eigenvector.items():
    eigenvector[key] = value / max_eigen

for k,v in sorted(eigenvector.items(), key=lambda p:p[1], reverse=True):
    print(k,v)

steam 1.0
isthisyoshide 0.997313117743019
iberrykarsen 0.34126603333013417
valvesoftware 0.2517032914307647
facikwacik 0.10840290439177311
faceit 0.08556440520743382
redlix_tf 0.04257428909573612
n0lansn 0.03988740683875504
dude_stay 0.033170201196302256
coffegrand 0.027796436682340116
shorktf2 0.017944535073409314
dota2 0.013914211687937815
valvecorp 0.013914211687937815
natabaptist 0.013018584268944117
tacowithapen 0.013018584268944117
nyahnyahnikkini 0.008988260883472474
mrgameandplay2 0.008092633464478784
khaki_jeanart 0.00630137862649139
osoponso 0.0054057512074976895
crazycold739 0.004510123788503987
derdiedasjan136 0.004510123788503987
zoomviex 0.004510123788503987
thee_kopje 0.004510123788503987
seriel18 0.004510123788503987
system64exe 0.004510123788503987
teamfortress 0.004510123788503987
nawaminsiam 0.004510123788503987
fooled_frog 0.004510123788503987
filippecak 0.003614496369510285
time5747 0.003614496369510285
ritzin3 0.003614496369510285
redsoney 0.003614496369510285
pog